In [21]:
import configparser
import psycopg2

In [22]:
from sql_queries import create_table_queries, drop_table_queries, copy_table_queries, insert_table_queries


In [23]:
def drop_tables(cur, conn):
    """
    This drops the existing Redshift tables (if applicable)
    """
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [24]:
def create_tables(cur, conn):
    """
    Creates the tables if they do not already exist
    """
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [27]:
def main():
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()

    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()

In [32]:
if __name__ == "__main__":
    main()

In [29]:
def load_staging_tables(cur, conn):
    """
    This loads the S3 files into staging tables in Redshift
    """
    for query in copy_table_queries:
        cur.execute(query)
        conn.commit()

In [30]:
def insert_tables(cur, conn):
    """
    This loads the data to the fact and dimension tables
    from each query
    """
    for query in insert_table_queries:
        cur.execute(query)
        conn.commit()

In [31]:
def main():
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
    
    load_staging_tables(cur, conn)
    insert_tables(cur, conn)